In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb
import lightgbm as lgb
import warnings
from sklearn.metrics import accuracy_score, confusion_matrix

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)

# Read Data

In [ ]:
df_train = pd.read_csv('../input/forest-cover-type-prediction/train.csv')
df_test = pd.read_csv('../input/forest-cover-type-prediction/test.csv')
df_train.drop(['Id'], axis=1, inplace=True)
y = df_train.iloc[:, -1]
df_train = df_train.iloc[:, :-1]
test_Id = df_test.iloc[:, 0]
df_test.drop(['Id'], axis=1, inplace=True)

# Take a look at the data

In [ ]:
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
all_data = pd.concat([df_train, df_test], axis=0, sort=False)
print(ntrain, ntest)
print(all_data.shape)
all_data.head()

In [ ]:
df_train.head()

In [ ]:
df_train.describe().T

## Drop useless columns

In [ ]:
df_train.drop(['Soil_Type7', 'Soil_Type15'], axis=1, inplace=True)

In [ ]:
sns.boxplot('Horizontal_Distance_To_Hydrology', data=all_data)

In [ ]:
# Checking correlatinos

In [ ]:
corrmat = all_data.iloc[:, :10].corr()
plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat)

From the above corr graph we see that:

1- Hilshade at 3pm and aspect ratio are correlated

2- Horizontal_Distance_To_Hydrology and Vertical_Distance_To_Hydrology are correlated

In [ ]:
plt.subplots(figsize=(12, 8))
plt.subplot(1, 2, 1)
sns.scatterplot(x='Horizontal_Distance_To_Hydrology', y='Vertical_Distance_To_Hydrology', data=all_data)
plt.subplot(1, 2, 2)
sns.scatterplot(x='Hillshade_3pm', y='Aspect', data=all_data)

# Adding New Features

##### We now create a new feature distance to hydrology = sqrt((Horizontal_Distance_To_Hydrology)^2 + (Vertical_Distance_To_Hydrology)^2). We will multiply with sign of vertical distance since it represents whether it is above or below. We also create few other variables.

In [ ]:
a = all_data['Horizontal_Distance_To_Hydrology']
b = all_data['Vertical_Distance_To_Hydrology']
all_data['Distance_To_Hydrology'] = np.sqrt(np.power(a, 2) + np.power(b, 2))

In [ ]:
all_data['Horizontal_distance'] = (all_data['Horizontal_Distance_To_Hydrology'] + 
                                   all_data['Horizontal_Distance_To_Roadways']  + 
                                   all_data['Horizontal_Distance_To_Fire_Points']) /3

In [ ]:
all_data['average_hillshade'] = (all_data['Hillshade_3pm'] + 
                                 all_data['Hillshade_Noon'] + 
                                 all_data['Hillshade_9am'])/3

In [ ]:
# high negative correlation, therefore making new feature
all_data['Aspect_hillshade'] = (all_data['Aspect'] * all_data['Hillshade_9am'])/255
all_data['slope_hillshade'] = (all_data['Slope'] * all_data['Hillshade_Noon'])/255
all_data['Elevation'] = [math.floor(v/50.0) for v in all_data['Elevation']]

In [ ]:
all_data['EHDtH'] = all_data['Elevation'] - all_data['Horizontal_Distance_To_Hydrology'] * 0.2

In [ ]:
all_data['Distanse_to_Hydrolody'] = (all_data['Horizontal_Distance_To_Hydrology'] ** 2 + 
                                     all_data['Vertical_Distance_To_Hydrology'] ** 2) ** 0.5

In [ ]:
all_data['Hydro_Fire_1'] = (all_data['Horizontal_Distance_To_Hydrology'] + 
                            all_data['Horizontal_Distance_To_Fire_Points'])

In [ ]:
all_data['Hydro_Fire_2'] = abs(all_data['Horizontal_Distance_To_Hydrology'] - 
                               all_data['Horizontal_Distance_To_Fire_Points'])

In [ ]:
all_data['Hydro_Road_1'] = abs(all_data['Horizontal_Distance_To_Hydrology'] + 
                               all_data['Horizontal_Distance_To_Roadways'])

In [ ]:
all_data['Hydro_Road_2'] = abs(all_data['Horizontal_Distance_To_Hydrology'] - 
                               all_data['Horizontal_Distance_To_Roadways'])

In [ ]:
all_data['Fire_Road_1'] = abs(all_data['Horizontal_Distance_To_Fire_Points'] + 
                              all_data['Horizontal_Distance_To_Roadways'])

In [ ]:
all_data['Fire_Road_2'] = abs(all_data['Horizontal_Distance_To_Fire_Points'] - 
                              all_data['Horizontal_Distance_To_Roadways'])

In [ ]:
all_data.head()

In [ ]:
num_labels = [i for i in all_data.columns[0:10]]
b = ['distance_to_hydrology','Horizontal_distance',
     'average_hillshade','Aspect_hillshade','slope_hillshade','EVDtH',
      'EHDtH','Distanse_to_Hydrolody', 'Hydro_Fire_1','Hydro_Fire_2','Hydro_Road_1', 
     'Hydro_Road_2','Fire_Road_1','Fire_Road_2']
num_labels.extend(b)

In [ ]:
train_data = all_data.iloc[:df_train.shape[0],:]

test_data = all_data.iloc[df_train.shape[0]:,:]

# Handling Outliers doesn't give any performance improvement

In [ ]:
# Outlier treatment for only continuous variables
'''
for col in num_labels:
    percentiles = train_data[col].quantile([0.01,0.99]).values
    train_data[col][train_data[col] <= percentiles[0]] = percentiles[0]
    train_data[col][train_data[col] >= percentiles[1]] = percentiles[1]
    test_data[col][test_data[col] <= percentiles[0]] = percentiles[0]
    test_data[col][test_data[col] >= percentiles[1]] = percentiles[1]
'''

# Scaling

In [ ]:
rs = RobustScaler()
rs.fit(train_data)
train_data = rs.transform(train_data)
test_data = rs.transform(test_data)
train_data = pd.DataFrame(train_data, columns = all_data.columns)
test_data = pd.DataFrame(test_data, columns = all_data.columns)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

# Handling Skewness doesn't give any performance improvement

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size = 0.1, shuffle= True)

# Modelling

In [ ]:
etc = ExtraTreesClassifier(n_estimators=400)
etc.fit(X_train, y_train)

In [ ]:
u = etc.predict(X_test)
print(accuracy_score(u, y_test))
print(cross_val_score(etc, X_test, y_test, cv=5).mean())

In [ ]:
etc.fit(train_data, y)
test_predictions = etc.predict(test_data)

In [ ]:
solutions = pd.DataFrame({'Id': test_Id, 'Cover_Type': test_predictions})
solutions.to_csv('finsubmission.csv',index=False)